In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

%matplotlib inline
import matplotlib.pyplot as plt

#import matplotlib
#matplotlib.use('agg')
#import matplotlib.pyplot as plt 

from PIL import Image
from matplotlib.pyplot import imshow
import matplotlib.cm as cm

import numpy as np

import skimage.transform
from scipy import ndimage
import scipy.misc

def image_show(image_path):
  imshow(np.asarray(Image.open(image_path, 'r')))
  plt.axis('off')
  plt.show()

import tensorflow as tf
flags = tf.app.flags
FLAGS = flags.FLAGS

#flags.DEFINE_string('model_dir', '/home/gezi/new/temp/image-caption/ai-challenger/model/showattentell.finetune/', '')
  
import sys, os, math
import gezi, melt
import numpy as np

from deepiu.util import algos_factory
from deepiu.seq2seq.rnn_decoder import SeqDecodeMethod

#debug
from deepiu.util import text2ids

TEXT_MAX_WORDS = 100    
decode_max_words = 20


from deepiu.util import ids2text
vocab_path = '/home/gezi/new/temp/image-caption/ai-challenger/tfrecord/seq-basic-finetune/vocab.txt'
ids2text.init(vocab_path)

image_dir = image_dir = '/home/gezi/data2/data/ai_challenger/image_caption/pic/'
image_file = '6275b5349168ac3fab6a493c509301d023cf39d3.jpg'
image_path = os.path.join(image_dir, image_file)
image_model_checkpoint_path = '/home/gezi/data/image_model_check_point/inception_resnet_v2_2016_08_30.ckpt'
image_model_name='InceptionResnetV2'
FLAGS.model_dir = model_dir = '/home/gezi/new/temp/image-caption/ai-challenger/model/showattentell.finetune/epoch/model.ckpt-25.3-516000'
#FLAGS.model_dir = model_dir = '/home/gezi/new/temp/image-caption/ai-challenger/model/showattentell.finetune/epoch/model.ckpt-20.0-341406'
#FLAGS.model_dir = model_dir = '/home/gezi/new/temp/image-caption/ai-challenger/model/showattentell.coverage.finetune/epoch/'


if not melt.varname_in_checkpoint(image_model_name, model_dir):
  image_model = melt.image.ImageModel(image_model_checkpoint_path, 
                                      model_name=image_model_name)
else:
  image_model = None

import libpinyin
pinyin = libpinyin.Pinyin()
pinyin.Load('./data/pinyin/')

tensorflow_version: 1.2.0-rc0


INFO:tensorflow:Initializing vocabulary from file: /home/gezi/new/temp/image-caption/ai-challenger/tfrecord/seq-basic-finetune/vocab.txt
INFO:tensorflow:Created vocabulary with 10148 words


ENCODE_UNK 1


In [2]:
if 'coverage' in model_dir:
  FLAGS.coverage_attention_wrapper = 1

FLAGS.image_encoder = 'Rnn'
FLAGS.image_attention_size = 64 
FLAGS.image_endpoint_feature_name = 'Conv2d_7b_1x1'
FLAGS.image_model = 'InceptionResnetV2'
FLAGS.image_checkpoint_file = '/home/gezi/data/image_model_check_point/inception_resnet_v2_2016_08_30.ckpt' 
FLAGS.pre_calc_image_feature = 0 
FLAGS.finetune_image_model = 1 
FLAGS.image_features_batch_norm = 1 
FLAGS.image_features_drop_out = 1
FLAGS.emb_dim = 512 
FLAGS.rnn_hidden_size = 512 
FLAGS.image_feature_len = 1536 * 64

beam_size = 10
length_normalization_factor = 0.25

In [3]:
algo = 'show_and_tell'
global_scope = algo
main_scope = 'main'
melt.apps.image_processing.init()
with tf.variable_scope(global_scope):
  with tf.variable_scope(main_scope):
    predictor =  algos_factory.gen_predictor(algo)
  
    beam_text, beam_text_score = predictor.init_predict_text(decode_method=SeqDecodeMethod.ingraph_beam, 
                                                        beam_size=beam_size,
                                                        length_normalization_factor=length_normalization_factor)  
    predictor.beam_text, predictor.beam_text_score = beam_text, beam_text_score

predictor.load(FLAGS.model_dir) 

attention_option: luong
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.


cell: <class 'tensorflow.python.ops.rnn_cell_impl.BasicLSTMCell'>
cell: <class 'tensorflow.python.ops.rnn_cell_impl.BasicLSTMCell'>
rnn decoder gen only mode


image_model will get feature_name Conv2d_7b_1x1
image_feature: Tensor("show_and_tell/main/main/Flatten/Reshape:0", shape=(?, 98304), dtype=float32)
---------- Tensor("show_and_tell/main/main_1/concat_2:0", shape=(?, 1792), dtype=float32)
---------- Tensor("show_and_tell/main/main_1/Reshape_1:0", shape=(?, 30), dtype=int32) Tensor("show_and_tell/main/main_1/Gather:0", shape=(?, 10, 30), dtype=int32) Tensor("show_and_tell/main/main_1/TopKV2:0", shape=(?, 10), dtype=float32) 30
INFO:tensorflow:Restoring parameters from /home/gezi/new/temp/image-caption/ai-challenger/model/showattentell.finetune/epoch/model.ckpt-25.3-516000
restore ok: /home/gezi/new/temp/image-caption/ai-challenger/model/showattentell.finetune/epoch/model.ckpt-25.3-516000


load model ok /home/gezi/new/temp/image-caption/ai-challenger/model/showattentell.finetune/epoch/model.ckpt-25.3-516000 duration: 7.60149407387


In [4]:
def predict(image_path, predictor):
  timer = gezi.Timer('beam search using time')
  img = melt.read_image(image_path)
  if image_model:
    #attention model gen features only
    feature = image_model.gen_features(img) if image_model is not None else img
  else:
    feature = [img]    

  texts_list, scores_list = predictor.predict_text(feature)
  texts = texts_list[0]
  scores = scores_list[0]
  print(texts[0], len(texts[0]))
  for text, score in zip(texts, scores):
    print(ids2text.ids2text(text), score)
  timer.print()

In [5]:
predict(image_path, predictor)

[ 748   21   19    3   10   44   18    3 1876    1   14    9    9    9    9
    9    9    9    9    9    9    9    9    9    9    9    9    9    9    9] 30
田地 里 站 着 一个 双手 拿 着 渔网 的 男人 </S> 0.0243456
田地 里 站 着 一个 右手 拿 着 渔网 的 男人 </S> 0.0174184
宽敞 的 田地 里 站 着 一位 双手 拿 着 渔网 的 男士 </S> 0.0140581
宽敞 的 田地 里 站 着 一位 双手 拿 着 网兜 的 男士 </S> 0.0139881
宽阔 的 田地 里 站 着 一个 右手 拿 着 渔网 的 男人 </S> 0.013862
宽阔 的 田地 里 站 着 一个 双手 拿 着 渔网 的 男人 </S> 0.012865
田地 里 站 着 一个 右手 拿 着 工具 的 男人 </S> 0.0109911
一个 双手 拿 着 渔网 的 男人 站 在 池塘 里 </S> 0.0102351
一个 双手 拿 着 渔网 的 男人 站 在 池塘 边 </S> 0.00838055
一个 双手 拿 着 渔网 的 男人 站 在 河边 的 石头 上 </S> 0.00826508


beam search using time duration: 1.67791080475


In [ ]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:10000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [ ]:
while True:
  image_name = raw_input('image_name like 6275b5349168ac3fab6a493c509301d023cf39d3.jpg:')
  image_name = image_name.strip()
  if image_name == 'q':
    break
  if not image_name.endswith('.jpg'):
    image_name += '.jpg'

  image_path = os.path.join(image_dir, image_name)

  if not os.path.exists(image_path):
    print('path not exists:%s'%image_path)
    image_path = image_name
    if not os.path.exists(image_path):
        continue
  plt.subplot(1, 1, 1)
  image_show(image_path)
  predict(image_path, predictor)
  #predict(image_path, predictor2, gen_feature=False)

In [ ]:
import glob 
for img in glob.glob('/home/gezi/new2/data/ai_challenger/image_caption/test_pic/*.jpg'):
    image_show(img)
    predict(img, predictor)